In [11]:
#
# Import libraries
#

import numpy as np
import os
import pandas as pd
import sys

from itertools import cycle

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler

from IPython.display import Image


#
# Import own libraries
#
utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

from myLog import myLog
from PlotEDA import PlotEDA
from PlotMV import PlotMV
from PCA_UMAP import PCA_UMAP, PCA_Var

In [12]:
#
# Constants
#

MVF_thr = 0.2
MVO_thr = 0.1

In [13]:
working_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA'

xq_path = os.path.join(working_path, 'OriginalFiles', 'RBR_Xq_minus_X.xlsx')
sn2tmtc_path = os.path.join(working_path, 'OriginalFiles', 'Seqn_TMT_Cohorte.xlsx')

fileSummary = os.path.join(working_path, 'WorkingFiles', 'Plots', 'SummaryPlots.html')
filePCA = os.path.join(working_path, 'WorkingFiles', 'Plots', 'PCAPlots.html')
if os.path.exists(fileSummary): os.remove(fileSummary)
if os.path.exists(filePCA): os.remove(filePCA)

In [14]:
mdata = pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\PESA\WorkingFiles\main_metadata.tsv', sep='\t')

In [15]:
#
# Set logging
#

logw = myLog(os.path.join(working_path, 'WorkingFiles', 'info.log'))
logw('Start Session')

Start Session


In [16]:
#
# Read files
#

xq = pd.read_excel(xq_path, header=[0,1])
sn2tmt = pd.read_excel(sn2tmtc_path)

In [17]:
xq = xq.rename(columns={'Proteins': 'qdesc', 'Accession': 'fid'})

In [18]:
#
# Create q2info.tsv
#

q2i = xq['q'].loc[:, ['fid', 'qdesc', 'Np']]
q2i.to_csv(os.path.join(working_path, 'WorkingFiles', 'q2info.tsv'), sep='\t', index=False)

In [19]:
xq = xq.drop(['Np', 'qdesc'], level=1, axis=1)

In [20]:
#
# Create Xq_minus_X.tsv
#

xq = pd.merge(
    xq.set_index(xq[('q', 'fid')]).drop('q', level=0, axis=1).T.reset_index(names=['Cohorte', 'TMT']),
    sn2tmt.loc[:, ['Cohorte', 'TMT', 'Seqn']],
    how='outer',
    on=['Cohorte', 'TMT']
).drop(['Cohorte', 'TMT'], axis=1).set_index('Seqn')

xq.to_csv(os.path.join(working_path, 'WorkingFiles', 'Xq_minus_X.tsv'), sep='\t', index=False)

In [21]:
#
# Start Filtering, Standardization, Missing Value imputation, and Batch Effect analysis
#

In [22]:
#
# Check Missing Values
#

plotMV = PlotMV(xq, mdata, file=fileSummary)
plotMV.plotSummary()

In [23]:
logw(f"Total number of observations: {xq.shape[0]}")
logw(f"Total number of proteins: {xq.shape[1]}")
logw(f"Total number of proteins with <{MVF_thr*100}% of missing values(<{int(xq.shape[0]*MVF_thr)} of obs.): {((xq.isna().sum()/xq.shape[0])<=MVF_thr).sum()}")

#logw("Missing elements: No missing element")

Total number of observations: 444
Total number of proteins: 6213
Total number of proteins with <20.0% of missing values(<88 of obs.): 470


In [24]:
#
# Generate Xq_minus_X_norm.tsv
# 

# Filter by 0.2 missing values
xqf = xq.loc[:, (xq.isna().sum()/xq.shape[0])<MVF_thr].copy()

In [25]:
#
# Filter observations by Missing values
#

plotMV = PlotMV(xqf, mdata, file=fileSummary)
plotMV.plotSummaryObs()

# Filter Observations by missing values

xqf = xqf[xqf.isna().sum(axis=1)/xqf.shape[1]<MVO_thr]

logw(f'Total number of observations with <{MVO_thr*100}% of missing values: {xqf.shape[0]} / {xq.shape[0]}')

Total number of observations with <10.0% of missing values: 424 / 444


In [26]:
#
# Center and scale
#

xqfn = pd.DataFrame(
    StandardScaler().fit_transform(xqf),
    columns=xqf.columns, index=xqf.index
)

In [27]:
#
# Summary plots
#

plotEDA = PlotEDA(xqf, mdata, file=fileSummary)

plotEDA.plotSummary(r11=(-0.4,0.4), r12=(0,2), r21=(-0.4,0.4), r22=(0,2), r3=(-4,4),vl3=[0])
plotEDA.plotByGroup('Group', vl1=[0],vl2=[0])
plotEDA.plotByGroup('Cohort',vl1=[0],vl2=[0])

In [28]:
#
# Imputation of missing values using KNN
#

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
xqfnv = pd.DataFrame(
    imputer.fit_transform(xqfn),
    columns=xqfn.columns,
    index=xqfn.index
)


In [29]:
# Number of imputed missing values

logw(f"Total number of values after filtration: {xqf.shape[0]*xqf.shape[1]}")
logw(f"Number of imputed missing values: {xqf.isna().sum().sum()} ({round(100*xqf.isna().sum().sum()/(xqf.shape[0]*xqf.shape[1]),2)}%)")

Total number of values after filtration: 199280
Number of imputed missing values: 6024 (3.02%)


In [30]:
catVars = ['Group', 'Smoke']
conVars = ['Calcium_Score', 'HDL', 'LDL', 'Total_Cholesterol','Ox-LDL','Lipoprotein(a)','CRP', 'Plaque_thickness']

In [31]:
#
# Correct batch effect using combat
#
# https://github.com/brentp/combat.py
#

# from combat import combat

# xqfnvb = combat(
#     data=xqfnv.T,
#     batch=mdata.set_index('Seqn').loc[xqf.index, 'Cohort']
# ).T


from myComBat import myComBat

xqfnvb = myComBat(xqfnv, mdata, 'Cohort', catVars, conVars, Rpath=os.path.join(working_path, 'WorkingFiles', 'myRData'))

Loading required package: mgcv
Loading required package: nlme
This is mgcv 1.8-41. For overview type 'help("mgcv-package")'.
Loading required package: genefilter
Loading required package: BiocParallel
Warning messages:
1: package 'sva' was built under R version 4.2.1 
2: package 'mgcv' was built under R version 4.2.2 
3: package 'nlme' was built under R version 4.2.2 
4: package 'genefilter' was built under R version 4.2.2 
5: package 'BiocParallel' was built under R version 4.2.2 
Found4batches
Adjusting for12covariate(s) or covariate level(s)
Standardizing Data across genes
Fitting L/S model and finding priors
Finding nonparametric adjustments
Adjusting the Data




In [32]:
plotEDA = PlotEDA(xqfnvb, mdata, file=fileSummary)
plotEDA.plotByGroup('Group', vl1=[0],vl2=[0], plotN=False, titleLabel='- Corrected by Batch Effect')
plotEDA.plotByGroup('Cohort',vl1=[0],vl2=[0], plotN=False, titleLabel='- Corrected by Batch Effect')

In [33]:
ploteda = PlotEDA(xqfnvb, mdata, file=fileSummary)
ploteda.plotSummary(plots=[1], titleLabel='- Corrected by Batch Effect', r11=(-0.5, 0.5), r12=(0.85, 1.15))

In [34]:
#
# Kruskal-Wallisk H test 
#

logw('')
logw('Kuskal-Wallis test')
logw('Non corrected')
logw(f'Group - {plotEDA._kruskal(xqfnv, "Group")}')
logw(f'Cohort - {plotEDA._kruskal(xqfnv, "Cohort")}')
logw('Corrected')
logw(f'Group - {plotEDA._kruskal(xqfnvb, "Group")}')
logw(f'Cohort - {plotEDA._kruskal(xqfnvb, "Cohort")}')


Kuskal-Wallis test
Non corrected
Group - KruskalResult(statistic=7.295921026031224, pvalue=0.006911133205515113)
Cohort - KruskalResult(statistic=76.52391788736688, pvalue=1.7078472513712089e-16)
Corrected
Group - KruskalResult(statistic=6.258027115254663, pvalue=0.012363180779176308)
Cohort - KruskalResult(statistic=0.6730481351260096, pvalue=0.8795242346430892)


In [35]:
xqfnvb.to_csv(os.path.join(working_path, 'WorkingFiles', 'Xq_minus_X_norm.tsv'), sep='\t', index=True)

In [36]:
pcaumap = PCA_UMAP(xqfnv, mdata, file=filePCA)
pcaumap.plotReduction('Cohort', pcacomp=[0,1])
pcaumap.plotReduction('Group', pcacomp=[0,1])

In [37]:
pcaumap = PCA_UMAP(xqfnvb, mdata, file=filePCA)
pcaumap.plotReduction('Cohort', pcacomp=[0,1], titleLabel='- Corrected by Batch Effect')
pcaumap.plotReduction('Group', pcacomp=[0,1], titleLabel='- Corrected by Batch Effect')

In [38]:
PCA_Var(xqfnv, mdata, conVars, catVars+['Cohort'], n_comp=10)

,%Var PCA,Calcium_Score,HDL,LDL,Total_Cholesterol,Ox-LDL,Lipoprotein(a),CRP,Plaque_thickness,Group,Smoke,Cohort
1,11.428637,2.784549e-01,6.409470e-01,0.273595,7.527742e-02,7.218451e-02,0.422157,6.816749e-01,0.041296,0.011915,0.194908,1.979417e-01
2,6.800875,7.530169e-01,1.159864e-01,0.014129,1.187416e-03,2.486087e-04,0.495967,2.025507e-02,0.154240,0.013401,0.041719,3.723152e-01
3,4.136326,3.431212e-02,2.194270e-01,0.417542,6.936876e-01,4.827007e-01,0.061774,1.577544e-04,0.196578,0.277653,0.073204,1.538343e-03
4,2.954340,4.457627e-05,6.572315e-02,0.456422,3.239683e-01,1.323138e-01,0.650488,4.849171e-01,0.292248,0.323697,0.602156,2.453304e-02
5,2.110144,4.196560e-01,5.450338e-10,0.028476,1.502604e-01,1.928039e-01,0.971979,5.096182e-10,0.030755,0.038776,0.029869,2.429616e-01
6,1.774960,2.945097e-03,2.271334e-05,0.636740,4.701854e-01,2.860553e-01,0.434675,1.513644e-13,0.078010,0.218436,0.015265,9.819606e-11
7,1.746863,6.888004e-03,7.833515e-01,0.080082,1.349858e-02,1.479138e-02,0.087495,8.605108e-01,0.048459,0.951061,0.000997,3.024800e-09
8,1.514122,5.551612e-01,1.402630e-05,0.565639,5.671260e-01,1.126160e-01,0.033897,2.293207e-10,0.003147,0.038401,0.063875,1.647162e-01
9,1.405869,3.239594e-02,8.314667e-01,0.002632,6.029339e-09,6.304026e-08,0.888753,8.150418e-01,0.562716,0.697956,0.488609,4.428902e-07
10,1.361567,8.958535e-07,1.751845e-01,0.888256,3.115513e-01,8.710914e-03,0.043011,1.157430e-03,0.077105,0.048164,0.987345,2.506219e-08


In [39]:
xqfnvb = pd.read_csv(os.path.join(working_path, 'WorkingFiles', 'Xq_minus_X_norm.tsv'), sep='\t', index_col='Seqn')

In [40]:
PCA_Var(xqfnvb, mdata, conVars, catVars+['Cohort'], n_comp=10).T

,1,2,3,4,5,6,7,8,9,10
%Var PCA,9.486122,6.751791,4.024290,2.767219,2.121711e+00,1.836975,1.691666e+00,1.587290e+00,1.381879,1.344555
Calcium_Score,0.086204,0.770956,0.025581,0.000117,3.071057e-01,0.445233,4.766037e-02,1.700421e-01,0.005984,0.000937
HDL,0.385950,0.233479,0.048520,0.341560,1.661598e-13,0.160277,2.529151e-05,2.643770e-02,0.785151,0.444516
LDL,0.036930,0.096086,0.418738,0.454769,6.251421e-03,0.504709,3.171920e-02,2.579116e-01,0.477391,0.870896
Total_Cholesterol,0.002627,0.042899,0.658051,0.287835,2.847695e-02,0.870821,5.529377e-05,6.344648e-04,0.344744,0.423536
Ox-LDL,0.001203,0.016935,0.316810,0.177248,2.925589e-03,0.569997,4.983156e-02,3.146647e-07,0.834523,0.794788
Lipoprotein(a),0.880633,0.377564,0.076972,0.897325,4.532393e-01,0.500301,6.316735e-01,2.646867e-01,0.250619,0.050146
CRP,0.415539,0.020830,0.000088,0.600963,1.880710e-16,0.007732,6.946226e-11,5.359135e-02,0.025526,0.017179
Plaque_thickness,0.016218,0.697681,0.068807,0.244265,1.358161e-02,0.407032,3.649530e-03,1.219368e-01,0.392546,0.692790
Group,0.001232,0.332922,0.214763,0.318992,1.444513e-02,0.905384,8.825276e-02,8.687110e-02,0.143828,0.525050
